<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/KAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Test

In [ ]:
!rm -r /content/data/knowledge_embeddings

In [44]:
!cp -r /content/data/knowledge_embeddings /content/drive/MyDrive/data

In [46]:
!ls /content/data/knowledge_embeddings/7006.npy

/content/data/knowledge_embeddings/7006.npy


In [48]:
!ls /content/drive/MyDrive/data/knowledge_embeddings/7006.npy

/content/drive/MyDrive/data/knowledge_embeddings/7006.npy


In [ ]:
!unzip -uq '/content/drive/MyDrive/data_path.zip' -d '/content/drive/MyDrive/data_path'

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data import DataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks

In [ ]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)

# Download and unzip data

In [ ]:
!cp /content/drive/MyDrive/files/data_path.zip /content
!unzip data_path.zip "data/jsons/*" "data/pic/*" "data/ques_embeddings/bioelmo/*" "data/answer_word_frequency.csv" "data/answer_word_frequency.xlsx" -d /content

In [2]:
!mkdir data
!cp /content/drive/MyDrive/data/jsons/QA.json /content/data
!cp /content/drive/MyDrive/data/knowledge.txt /content/data

In [3]:
!cp /content/drive/MyDrive/data/knowledge.txt /content/data

# Object detection

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# test installation
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
!cp /content/models/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb /content/drive/MyDrive/Colab\ Notebooks

# Entity extraction

## test

In [ ]:
# The textual entities are KG
# entities associated with key phrases of the question, where we utilize the Stanford NLP Dependency Parser [4] and Stanford Named
# Entity Recognizer [5] to analyze the question and extract all noun
# phrases, verb-object phrases and named entities as the key phrases.


In [ ]:
from sklearn.utils import shuffle
import pandas as pd
import pathlib

root = pathlib.Path('/content/data')
d = shuffle(pd.read_json(root.joinpath('jsons/open_ended').with_suffix('.json')), random_state=1)

In [ ]:
dq = d['Questions']
print(dq)

12134                                     What is present?
9969                                      What is present?
3785                                     What is present ?
2947                            What does this image show?
858                     What are colors of these hormones?
                               ...                        
7813                                      What is present?
10955                           What does this image show?
5192                            Where does this belong to?
12172                                     What is present?
235      How many copies does the red probe hybridize t...
Name: Questions, Length: 16429, dtype: object


In [ ]:
print(dq[0])

What are positively charged,  thus allowing the compaction of the negatively charged DNA?


In [ ]:
pip install stanfordcorenlp

## Extraction Method

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def ectract_query(ques):
  qs = nlp(ques)
  queries = []
  for tk in qs:
    # print(tk.text,'|',tk.pos_)
    if tk.pos_ == 'VERB' or tk.pos_ == 'NOUN' or tk.pos_ == 'ADJ':
      queries.append(tk.lemma_)
  return queries


# print(ectract_query(dq[0]))

In [ ]:
def knowledge_retrieval(queries):
  kns = []
  for q in queries:
    kns += query_knowledge(q)
  return(kns)

# queries = ectract_query(dq[0])
# query_knowledge

# Knowledge Embeddings

In [4]:
pip install transformers

     |████████████████████████████████| 2.1MB 13.0MB/s 
     |████████████████████████████████| 3.3MB 48.7MB/s 
     |████████████████████████████████| 870kB 44.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=77796ce8c1199554ca7879d99999ec882396e0467fd68de12c773ce4610e165f
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [6]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import json
import re
import os

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased", do_lower_case=True)
model = AutoModel.from_pretrained("bert-large-cased", output_hidden_states=True)

with open('/content/data/knowledge.txt','r') as file:
    lines = file.readlines()


# def preprocess_sentence(w):
#     w = w.lower().strip()
#     w = re.sub(r'[" "]+', " ", w)
#     w = re.sub(r"[^a-zA-Z ]", "", w)
#     w = w.strip()
#     return w


path = '/content/data/knowledge_embeddings'
if not os.path.exists(path):
    os.mkdir(path)


In [42]:
print(kns[756])

('757', ['heap manner of fill', 'frontal sinus is a sinus'])


In [10]:
kns = []
for l in lines:
  l_split = l.strip().split(',')
  kns.append((l_split[0],l_split[1:]))
print(len(kns))

32761


In [43]:
for kn in kns[757:]:
  ques_id = int(kn[0])
  ks = kn[1]
  k_emb = []
  for k in ks:
    k = '[CLS] ' + k + ' [SEP]'
    tokenized_text1 = tokenizer.tokenize(k) 
    indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)    
    segments_ids1 = [1] * len(tokenized_text1)  
    # shape: (1,17)  
    tokens_tensor1 = torch.tensor([indexed_tokens1])
    segments_tensors1 = torch.tensor([segments_ids1])
    with torch.no_grad():
      # Because we set 'output_hidden_states = True', the third item will be the
      # hidden states from all layers.
      outputs1 = model(tokens_tensor1, segments_tensors1)
      hidden_states1 = outputs1[2]
    
    # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
    token_embeddings1 = torch.stack(hidden_states1, dim=0)
    # Remove dimension 1, the "batches".
    # shape: (25,17,1024) - (layers,sentence length,embedding size)
    token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
    # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
    # shape: (17,25,1024) - (sentence length,layers,embedding size)
    token_embeddings1 = token_embeddings1.permute(1, 0, 2)
    token_vecs_sum1 = []

    # shape: (17,1024)
    for token in token_embeddings1:
        # Sum the vectors from the last four layers.
        sum_vec1 = torch.sum(token[-4:], dim=0)
        token_vecs_sum1.append(sum_vec1)
    
    
    k_emb += token_vecs_sum1
  # knowledge embedding for one question
  # shape: (27,1024)
  k_emb = np.array([t.numpy() for t in k_emb])
  # save
  np.save(os.path.join(path, str(ques_id)+'.npy'), k_emb)
  print("saved file: ", ques_id)
print("saved all")

流式输出内容被截断，只能显示最后 5000 行内容。
saved file:  2000
saved file:  2001
saved file:  2002
saved file:  2003
saved file:  2004
saved file:  2005
saved file:  2006
saved file:  2007
saved file:  2008
saved file:  2009
saved file:  2010
saved file:  2011
saved file:  2012
saved file:  2013
saved file:  2014
saved file:  2015
saved file:  2016
saved file:  2017
saved file:  2018
saved file:  2019
saved file:  2020
saved file:  2021
saved file:  2022
saved file:  2023
saved file:  2024
saved file:  2025
saved file:  2026
saved file:  2027
saved file:  2028
saved file:  2029
saved file:  2030
saved file:  2031
saved file:  2032
saved file:  2033
saved file:  2034
saved file:  2035
saved file:  2036
saved file:  2037
saved file:  2038
saved file:  2039
saved file:  2040
saved file:  2041
saved file:  2042
saved file:  2043
saved file:  2044
saved file:  2045
saved file:  2046
saved file:  2047
saved file:  2048
saved file:  2049
saved file:  2050
saved file:  2051
saved file:  2052
saved file:  2053
s

KeyboardInterrupt: ignored

In [ ]:
# What are positively charged,  thus allowing the compaction of the negatively charged DNA?
# what are positively charged thus allowing the compaction of the negatively charged dna
# [CLS] what are positively charged thus allowing the compaction of the negatively charged dna [SEP]
# ['[CLS]', 'what', 'are', 'positively', 'charged', 'thus', 'allowing', 'the', 'compact', '##ion', 'of', 'the', 'negatively', 'charged', 'd', '##na', '[SEP]']
# [101, 1184, 1132, 14257, 4601, 2456, 3525, 1103, 10114, 1988, 1104, 1103, 21598, 4601, 173, 1605, 102]
# [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# tensor([[  101,  1184,  1132, 14257,  4601,  2456,  3525,  1103, 10114,  1988,  1104,  1103, 21598,  4601,   173,  1605,   102]])
# tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


for qa in data:
  ques_id = qa['Question_Id']
  ques = qa['Questions']    

  # extract verbs and nouns from question
  queries = ectract_query(ques)
  # retrieve knowledge from knowledge base
  # num: len(queries) * 5 
  kns = knowledge_retrieval(queries)

  kn_concat = '[CLS]'
  for k in kns:
    marked_text1 = ' ' + k + " [SEP]"
    kn_concat += marked_text1   
  tokenized_text1 = tokenizer.tokenize(kn_concat)[:-1]  
  indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)    
  segments_ids1 = [1] * len(tokenized_text1)    
  tokens_tensor1 = torch.tensor([indexed_tokens1])
  segments_tensors1 = torch.tensor([segments_ids1])

  # tuple - shape: 25 * (1,17,1024) - (batch size,sentence length,embedding size)
  with torch.no_grad():
      # Because we set 'output_hidden_states = True', the third item will be the
      # hidden states from all layers.
      outputs1 = model(tokens_tensor1, segments_tensors1)
      hidden_states1 = outputs1[2]
  # Concatenate the tensors for all layers.
  # Use 'stack' to create a new dimension in the tensor.
  # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
  token_embeddings1 = torch.stack(hidden_states1, dim=0)
  # Remove dimension 1, the "batches".
  # shape: (25,17,1024) - (layers,sentence length,embedding size)
  token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
  # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
  # shape: (17,25,1024) - (sentence length,layers,embedding size)
  token_embeddings1 = token_embeddings1.permute(1, 0, 2)

  token_vecs_sum1 = []

  for token in token_embeddings1:
      # Sum the vectors from the last four layers.
      sum_vec1 = torch.sum(token[-4:], dim=0)
      token_vecs_sum1.append(sum_vec1)
  # shape: (17,1024)
  token_vec = np.array([t.numpy() for t in token_vecs_sum1])
  # kn_emds.append(token_vec)
  # kn_emds = np.concatenate(kn_emds)
  # Write .npy file to '/content/embedding/' folder.
  np.save(os.path.join(path, str(ques_id)+'.npy'), token_vec)
  print("saved file: ", ques_id)
print("saved all")

# # validate saved features and calculate max length of all questions
# max_len = 0
# for pair in data:
#     ques_id = pair['Question_Id']
#     emb = np.load('../data/ques_embeddings/bert/'+str(ques_id)+'.npy')
#     length = emb.shape[0]
#     if length >= max_len:
#         max_len = length
#         print('current max length is', max_len)

saved file:  1
saved file:  2
saved file:  3
saved file:  4
saved file:  5
saved file:  6
saved file:  7
saved file:  8
saved file:  9
saved file:  10
saved file:  11
saved file:  12
saved file:  13
saved file:  14
saved file:  15
saved file:  16
saved file:  17
saved file:  18
saved file:  19
saved file:  20
saved file:  21
saved file:  22
saved file:  23
saved file:  24
saved file:  25
saved file:  26
saved file:  27
saved file:  28
saved file:  29
saved file:  30
saved file:  31
saved file:  32
saved file:  33
saved file:  34
saved file:  35
saved file:  36
saved file:  37
saved file:  38
saved file:  39
saved file:  40
saved file:  41
saved file:  42
saved file:  43
saved file:  44
saved file:  45
saved file:  46
saved file:  47
saved file:  48
saved file:  49
saved file:  50
saved file:  51
saved file:  52
saved file:  53
saved file:  54
saved file:  55
saved file:  56
saved file:  57
saved file:  58
saved file:  59
saved file:  60
saved file:  61
saved file:  62
saved file:  63
s

RuntimeError: ignored

In [ ]:
# validate saved features and calculate max length of all questions
for pair in data:
  try:
    ques_id = pair['Question_Id']
    emb = np.load('/content/data/knowledge_embeddings/'+str(ques_id)+'.npy')
    # length = emb.shape[0]
    print(ques_id,'shape is', emb.shape, emb[1])        
  except:
    None

1 shape is (147, 1024) [ 2.0104558  -2.9711552  -2.076493   ... -0.87561285  1.5736442
  0.29449445]
2 shape is (112, 1024) [-1.3510771  -2.7878842  -0.26454362 ... -0.5927262   1.5895883
  2.508337  ]
3 shape is (234, 1024) [-1.3147087  -2.2564392  -0.45575547 ... -0.46712095  1.1960553
  2.5312965 ]
4 shape is (108, 1024) [ 2.2483888  -2.304226   -3.959382   ...  0.5228845   2.0124369
  0.39888477]
5 shape is (62, 1024) [ 1.3404918  -5.0190053  -0.80455905 ... -3.39945     1.167823
  1.1446658 ]
6 shape is (162, 1024) [ 0.6510856 -4.0878735 -0.8470171 ... -2.9745445  1.0813475  0.2658643]
7 shape is (193, 1024) [ 0.74279773 -4.3296885  -1.016265   ... -3.0224192   1.2225933
  0.3418101 ]
8 shape is (30, 1024) [-1.5323267 -3.1610942 -2.7585902 ... -3.5711322 -1.1927916  0.8481272]
9 shape is (186, 1024) [-0.62647927 -4.551471   -1.1530848  ... -1.5236076   1.7725874
  0.7414978 ]
10 shape is (111, 1024) [-0.1810945 -3.944694  -1.4811113 ... -1.2566606  1.9807792  0.5730399]
11 shape i

## BERT embeddings

In [ ]:
for qa in data:
    ques_id = qa['Question_Id']
    # What are positively charged,  thus allowing the compaction of the negatively charged DNA?
    ques = qa['Questions']
    # what are positively charged thus allowing the compaction of the negatively charged dna
    after = preprocess_sentence(ques)
  
    # [CLS] what are positively charged thus allowing the compaction of the negatively charged dna [SEP]
    marked_text1 = "[CLS] " + after + " [SEP]"
    # ['[CLS]', 'what', 'are', 'positively', 'charged', 'thus', 'allowing', 'the', 'compact', '##ion', 'of', 'the', 'negatively', 'charged', 'd', '##na', '[SEP]']
    tokenized_text1 = tokenizer.tokenize(marked_text1)
    # [101, 1184, 1132, 14257, 4601, 2456, 3525, 1103, 10114, 1988, 1104, 1103, 21598, 4601, 173, 1605, 102]
    indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    segments_ids1 = [1] * len(tokenized_text1)
    # tensor([[  101,  1184,  1132, 14257,  4601,  2456,  3525,  1103, 10114,  1988,  1104,  1103, 21598,  4601,   173,  1605,   102]])
    tokens_tensor1 = torch.tensor([indexed_tokens1])
    # tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
    segments_tensors1 = torch.tensor([segments_ids1])

    # tuple - shape: 25 * (1,17,1024) - (batch size,sentence length,embedding size)
    with torch.no_grad():
        # Because we set 'output_hidden_states = True', the third item will be the
        # hidden states from all layers.
        outputs1 = model(tokens_tensor1, segments_tensors1)
        hidden_states1 = outputs1[2]
    # Concatenate the tensors for all layers.
    # Use 'stack' to create a new dimension in the tensor.
    # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
    token_embeddings1 = torch.stack(hidden_states1, dim=0)
    # Remove dimension 1, the "batches".
    # shape: (25,17,1024) - (layers,sentence length,embedding size)
    token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
    # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
    # shape: (17,25,1024) - (sentence length,layers,embedding size)
    token_embeddings1 = token_embeddings1.permute(1, 0, 2)

    token_vecs_sum1 = []

    for token in token_embeddings1:
        # Sum the vectors from the last four layers.
        sum_vec1 = torch.sum(token[-4:], dim=0)
        token_vecs_sum1.append(sum_vec1)
    # shape: (17,1024)
    token_vec = np.array([t.numpy() for t in token_vecs_sum1])

    
#     # Write .npy file to '/content/embedding/' folder.
#     np.save(os.path.join(path, str(ques_id)+'.npy'), token_vec)
#     print("saved file: ", ques_id)
# # print("saved all")

# # validate saved features and calculate max length of all questions
# max_len = 0
# for pair in data:
#     ques_id = pair['Question_Id']
#     emb = np.load('../data/ques_embeddings/bert/'+str(ques_id)+'.npy')
#     length = emb.shape[0]
#     if length >= max_len:
#         max_len = length
#         print('current max length is', max_len)
    break

(17, 1024)


# Knowledge

## SenticNet

In [ ]:
pip install senticnet

     |████████████████████████████████| 51.9MB 142kB/s 


In [ ]:
from senticnet.senticnet import SenticNet

sn = SenticNet()
concept_info = sn.concept('love')
polarity_label = sn.polarity_label('love')
polarity_value = sn.polarity_value('love')
moodtags = sn.moodtags('love')
semantics = sn.semantics('love')
sentics = sn.sentics('love')

In [ ]:
print(concept_info)
print(polarity_label)
print(polarity_value)
print(moodtags)
print(semantics)
print(sentics)

{'polarity_label': 'positive', 'polarity_value': '0.83', 'sentics': {'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}, 'moodtags': ['#joy', '#eagerness'], 'semantics': ['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']}
positive
0.83
['#joy', '#eagerness']
['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']
{'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}


## ConceptNet

In [ ]:
pip install conceptnet-lite

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 2.3MB 6.2MB/s 
     |████████████████████████████████| 870kB 12.2MB/s 
  Created wheel for peewee: filename=peewee-3.14.4-cp37-cp37m-linux_x86_64.whl size=592992 sha256=8cd8bf9ec663042c9460c97d740cc036a591f3a5f76022641ec733d49abe6218
  Stored in directory: /root/.cache/pip/wheels/a0/ce/ff/732bd2a2729233b3a55bb5d8557d500f7be41bd21df3f01048
  Created wheel for lmdb: filename=lmdb-0.97-cp37-cp37m-linux_x86_64.whl size=219767 sha256=2f858ce7912fbc450bcd41a6f0157509aae35fb3247beb7f2376d650440052c6
  Stored in directory: /root/.cache/pip/wheels/7f/4d/44/b299a9b22d8eaaaa455304687fb6d4c02d9178c1530850e12a
Successfully built peewee lmdb
  Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99


In [ ]:
import conceptnet_lite

conceptnet_lite.connect("/path/to/conceptnet.db")

File not found: /path/to/conceptnet.db
Download compressed database


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[*] 0 bytes / 1.85 GB @ 0 bytes/s [------------------] [0.0%, 0s left]    [*] 2.5 MB / 1.85 GB @ 12.6 MB/s [------------------] [0.1%, 0s left]    [*] 20.4 MB / 1.85 GB @ 67.8 MB/s [------------------] [1.1%, 0s left]    [*] 37.8 MB / 1.85 GB @ 94.4 MB/s [------------------] [2.0%, 0s left]    [*] 55.5 MB / 1.85 GB @ 111.0 MB/s [------------------] [2.9%, 0s left]    [*] 73.3 MB / 1.85 GB @ 122.2 MB/s [------------------] [3.9%, 0s left]    [*] 91.2 MB / 1.85 GB @ 130.2 MB/s [------------------] [4.8%, 0s left]   

In [ ]:
from conceptnet_lite import Label, edges_for

def query_knowledge(q):
  try:
    d = {}
    for e in edges_for(Label.get(text=q, language='en').concepts, same_language=True):
      # print(e.start.text, "::", e.end.text, "|", e.relation.name, "|", e.etc['weight'])
      d[(e.start.text,e.relation.name,e.end.text)] = e.etc['weight']
      # print(e.etc['weight'])
    kn = []
    for i,f in enumerate(sorted(d.items(), key=lambda item: item[1], reverse=True)):
      # if i == 5:
      #   break
      k = ' '.join(f[0])
      k = k.replace('_', ' ')
      kn.append(k)
      # only one knowledge
      break
    return kn
  except:
    return []


In [ ]:
print(query_knowledge('edematous'))

['edematous synonym dropsical', 'antiedematous derived from edematous', 'edematously derived from edematous', 'nonedematous derived from edematous', 'edemateous form of edematous']
[]
[]


In [ ]:
from conceptnet_lite import Label

cat_concepts = Label.get(text='charge', language='en').concepts  
for c in cat_concepts:
    print("    Concept URI:", c.uri)
    print("    Concept text:", c.text)
    print("    Concept language:", c.language.name)